In [18]:
import os
import requests
import pandas as pd
import sqlite3

In [6]:
# download dos arquivos

# URL base para os datasets do IMDb
base_url = 'https://datasets.imdbws.com/'

# Listas de nomes de arquivos que você deseja baixar
arquivos=[
    'name.basics.tsv.gz',
    'title.akas.tsv.gz',
    'title.basics.tsv.gz',
    'title.crew.tsv.gz',
    'title.principals.tsv.gz',
    'title.episode.tsv.gz',
    'title.ratings.tsv.gz'
]

# Diretório de destino
destino_diretorio = 'data'

# Certifique-se de que o diretorio de destino existe
os.makedirs(destino_diretorio, exist_ok=True)

# Loop para baixar cada arquivo
for arquivo in arquivos:
    url = base_url + arquivo
    caminho_destino = os.path.join(destino_diretorio, arquivo)

    # Verifica se o arquivo já existe para evitar o download repetido
    if not os.path.exists(caminho_destino):
        print(f'Baixando {arquivo}.')
        response = requests.get(url)

        # Verifica se a solicitação foi bem-sucedida (código de status 200)
        if response.status_code == 200:
            with open(caminho_destino, 'wb') as f:
                f.write(response.content)
            print(f'{arquivo} baixado com sucesso')
        else:
            print(f'Falha ao baixar {arquivo}. Código de status: {response.status_code}')
    else:
        print(f'{arquivo} já existe. Pulando o download')
print("Download concluído.")

Baixando name.basics.tsv.gz.
name.basics.tsv.gz baixado com sucesso
Baixando title.akas.tsv.gz.
title.akas.tsv.gz baixado com sucesso
Baixando title.basics.tsv.gz.
title.basics.tsv.gz baixado com sucesso
Baixando title.crew.tsv.gz.
title.crew.tsv.gz baixado com sucesso
Baixando title.principals.tsv.gz.
title.principals.tsv.gz baixado com sucesso
Baixando title.episode.tsv.gz.
title.episode.tsv.gz baixado com sucesso
Baixando title.ratings.tsv.gz.
title.ratings.tsv.gz baixado com sucesso
Download concluído.


In [7]:
# Tratamento dos dados

# Diretórios
diretorio_dados = 'data'
diretorio_tratados = os.path.join(diretorio_dados,'tratados')

# Certifica-se de que o diretório "tratados" existe
os.makedirs(diretorio_tratados, exist_ok=True)

# Lista todos os arquivos no diretório "data"
arquivos = os.listdir(diretorio_dados)

# Loop para abrir, tratar e salvar cada arquivo
for arquivo in arquivos:
    caminho_arquivo = os.path.join(diretorio_dados, arquivo)

    if os.path.isfile(caminho_arquivo) and arquivo.endswith('.gz'):
        print(f'Lendo e tratando o arquivo {arquivo}.')

        # Lê o arquivo TSV usando o pandas
        df = pd.read_csv(caminho_arquivo, sep='\t', compression='gzip', low_memory=False)

        # Substitui os caracteres '\n' por um valor nulo
        df.replace({"\\N": None}, inplace=True)

        # Salva o DataFrame no diretorio "tratados" sem compressão"
        caminho_destino = os.path.join(diretorio_tratados, arquivo[:-3]) # remove a extenão .gz
        df.to_csv(caminho_destino, sep='\t', index=False)

        print(f'Tratamento concluído para {arquivo}. Arquivo tratado salvo em {caminho_destino}')

print("Todos os arquivos foram tratados e salvos no diretório 'tratados'.")


Lendo e tratando o arquivo name.basics.tsv.gz.
Tratamento concluído para name.basics.tsv.gz. Arquivo tratado salvo em data/tratados/name.basics.tsv
Lendo e tratando o arquivo title.akas.tsv.gz.
Tratamento concluído para title.akas.tsv.gz. Arquivo tratado salvo em data/tratados/title.akas.tsv
Lendo e tratando o arquivo title.basics.tsv.gz.
Tratamento concluído para title.basics.tsv.gz. Arquivo tratado salvo em data/tratados/title.basics.tsv
Lendo e tratando o arquivo title.crew.tsv.gz.
Tratamento concluído para title.crew.tsv.gz. Arquivo tratado salvo em data/tratados/title.crew.tsv
Lendo e tratando o arquivo title.episode.tsv.gz.
Tratamento concluído para title.episode.tsv.gz. Arquivo tratado salvo em data/tratados/title.episode.tsv
Lendo e tratando o arquivo title.principals.tsv.gz.
Tratamento concluído para title.principals.tsv.gz. Arquivo tratado salvo em data/tratados/title.principals.tsv
Lendo e tratando o arquivo title.ratings.tsv.gz.
Tratamento concluído para title.ratings.tsv.g

In [8]:
# validando dataframe
df = pd.read_csv('./data/tratados/name.basics.tsv', sep='\t')
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0027125,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0117057,tt0075213,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department","tt0054452,tt0049189,tt0057345,tt0056404"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0069467,tt0050976,tt0050986,tt0083922"


In [9]:
# Verifica a quantidade de dados nulos por coluna
df.isnull().sum()

nconst                      0
primaryName                 7
birthYear            12671560
deathYear            13052697
primaryProfession     2672147
knownForTitles        1565107
dtype: int64

In [10]:
# Salvando em banco de dados com SQLite

# Diretório
diretorio_tratados = os.path.join('data', 'tratados')
banco_dados = 'imdb_data.db'

# Conecta ao banco de dados SQLite
conexao = sqlite3.connect(banco_dados)

# Lista todos os arquivos no diretório "tratados"
arquivos = os.listdir(diretorio_tratados)

# Loop para ler cada arquivo e salvar em uma tabela SQLite
for arquivo in arquivos:
    caminho_arquivo = os.path.join(diretorio_tratados, arquivo)

    if os.path.isfile(caminho_arquivo) and arquivo.endswith(".tsv"):
        # Lê o arquivo TSV usando o pandas
        df = pd.read_csv(caminho_arquivo, sep='\t', low_memory=False)

        # Remove a extensão do nome do arquivo
        nome_tabela = os.path.splitext(arquivo)[0]

        # Substitui os caracteres especiais no nome da tebela
        nome_tabela = nome_tabela.replace('.', "_").replace("-", "_")

        # Salva o DataFrame na tabela SQLite
        df.to_sql(nome_tabela, conexao, index=False, if_exists='replace')

        print(f'Arquivo {arquivo} salvo como tabela {nome_tabela} no banco de dados.')
# Fecha a conexao com o banco de dados
conexao.close()

print("Todos os arquivos foram salvos no banco de dados")

Arquivo name.basics.tsv salvo como tabela name_basics no banco de dados.
Arquivo title.akas.tsv salvo como tabela title_akas no banco de dados.
Arquivo title.basics.tsv salvo como tabela title_basics no banco de dados.
Arquivo title.crew.tsv salvo como tabela title_crew no banco de dados.
Arquivo title.episode.tsv salvo como tabela title_episode no banco de dados.
Arquivo title.principals.tsv salvo como tabela title_principals no banco de dados.
Arquivo title.ratings.tsv salvo como tabela title_ratings no banco de dados.
Todos os arquivos foram salvos no banco de dados


In [23]:
# Nome do banco de dados
banco_dados = 'imdb_data.db'

# Conecta-se ao banco de dados SQLite
conexao = sqlite3.connect(banco_dados)

# Cria um cursor
cursor = conexao.cursor()

# Executa a consulta SQL para obter o nome das tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recupera os resultados da consulta
tabelas = cursor.fetchall()

# Exibe o nome das tabelas
print("Tabela no banco de dados:")
for tabela in tabelas:
    print(tabela[0])

# Fecha o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

Tabela no banco de dados:
name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
analitico_titulos
analitico_participantes


In [26]:
# Concta-se ao banco de dados SQLite
conexao = sqlite3.connect(banco_dados)

# Executa a consulta SQL para obter as 10 primeiras linhas da taela name_basics
query = """
SELECT 
  nconst,
  tconst,
  ordering,
  category
FROM title_basics 
LIMIT 10
"""
df = pd.read_sql_query(query, conexao)

# Fecha a conexão com o banco de dados
conexao.close()

In [27]:
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,None,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,None,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,None,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,None,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,None,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0.0,1894.0,None,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0.0,1894.0,None,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0.0,1894.0,None,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894.0,None,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0.0,1895.0,None,1,"Documentary,Short"


In [28]:
# Concta-se ao banco de dados SQLite
conexao = sqlite3.connect(banco_dados)

# Executa a consulta SQL para obter as 10 primeiras linhas da taela name_basics
query = """
SELECT 
  tp.nconst,
  tp.tconst,
  tp.ordering,
  tp.category,
  tb.genres
FROM title_principals tp

LEFT JOIN title_basics tb
  ON (tb.tconst = tp.tconst)

LIMIT 10
"""
df = pd.read_sql_query(query, conexao)

# Fecha a conexão com o banco de dados
conexao.close()

In [14]:
df

,nconst,tconst,ordering,category,genres
0,nm1588970,tt0000001,1,self,"Documentary,Short"
1,nm0005690,tt0000001,2,director,"Documentary,Short"
2,nm0374658,tt0000001,3,cinematographer,"Documentary,Short"
3,nm0721526,tt0000002,1,director,"Animation,Short"
4,nm1335271,tt0000002,2,composer,"Animation,Short"
5,nm0721526,tt0000003,1,director,"Animation,Comedy,Romance"
6,nm1770680,tt0000003,2,producer,"Animation,Comedy,Romance"
7,nm1335271,tt0000003,3,composer,"Animation,Comedy,Romance"
8,nm5442200,tt0000003,4,editor,"Animation,Comedy,Romance"
9,nm0721526,tt0000004,1,director,"Animation,Short"


In [15]:
analitico_titulos = """
CREATE TABLE IF NOT EXISTS analitico_titulos AS 

WITH 
participantes AS (
    SELECT 
        tconst,
        COUNT(DISTINCT nconst) AS qtParticipantes
    FROM title_principals 
    GROUP BY tconst
)

SELECT 
    tb.tconst,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.endYear,
    tb.genres,
    tr.averageRating,
    tr.numVotes,
    tp.qtParticipantes
FROM title_basics tb 

LEFT JOIN title_ratings tr
    ON (tr.tconst = tb.tconst)

LEFT JOIN participantes tp
    ON (tp.tconst = tb.tconst)

"""

analitico_participantes = """
CREATE TABLE IF NOT EXISTS analitico_participantes AS 

SELECT 
  tp.nconst,
  tp.tconst,
  tp.ordering,
  tp.category,
  tb.genres
FROM title_principals tp

LEFT JOIN title_basics tb
  ON (tb.tconst = tp.tconst)
"""

In [16]:
# lista de consultas
queries = [analitico_titulos, analitico_participantes]

for query in queries:

    # Diretórios
    banco_dados = "imdb_data.db"

    # Conecta ao banco de dados SQLite
    conexao = sqlite3.connect(banco_dados)

    # Consulta SQL para contar o número de pessoas participantes por título
    query = query

    # Executa a consulta SQL
    conexao.execute(query)

    # Fecha a conexao com o banco de dados
    conexao.close()

    print('Tabela criadas com sucesso.')

Tabela criadas com sucesso.
Tabela criadas com sucesso.
